<a href="https://colab.research.google.com/github/geoffwoollard/learn_cryoem_math/blob/master/nb/anm_in_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pycuda
!pip install scikit-cuda

     |████████████████████████████████| 1.7 MB 13.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 78 kB 8.9 MB/s 
     |████████████████████████████████| 69 kB 9.0 MB/s 
  Created wheel for pycuda: filename=pycuda-2021.1-cp37-cp37m-linux_x86_64.whl size=626634 sha256=c3b7ec6455cf2fc8e516bfb155df8be24c3b57052e86f1c8400810013c98def8
  Stored in directory: /root/.cache/pip/wheels/c4/ef/49/dc6a5feb8d980b37c83d465ecab24949a6aa19458522a9e001
  Created wheel for pytools: filename=pytools-2022.1.7-py2.py3-none-any.whl size=64772 sha256=a67e16027f1fa92e129441244ee230c2f499cf87e33704049166899283b8bbe3
  Stored in directory: /root/.cache/pip/wheels/24/c3/8d/9b093180175460a5fc613220b26027ae97cecbbf94fd3822af
Successfully built pycuda pytools
     |████████████████████████████████| 114 kB 14.7 MB/s 


In [3]:
import torch
from torch import tensor
import numpy as np

import pycuda.gpuarray as gpuarray
import pycuda.autoinit
from skcuda import linalg
linalg.init()

/usr/local/lib/python3.7/dist-packages/skcuda/cublas.py:284: UserWarning: creating CUBLAS context to get version number
  warnings.warn('creating CUBLAS context to get version number')


In [90]:
def hess_to_eig_gpu(hess,nmodes=3, shift=5):
    """
    hess, shape: 3*n_nodes, 3*n_nodes
    return 
      vec, shape 3*n_nodes,n_nodes
        first mode (column) is most dominant
    """
    assert max(hess.shape) < 50000, 'RAM maxes out ~ 20000'
    hess = np.array(hess, np.float32, order='F')
    hess_gpu = gpuarray.to_gpu(hess)
    vec_gpu, val_gpu = linalg.eig(hess_gpu, 'N', 'V')
    vec,val = vec_gpu.get().T, val_gpu.get()
    vec = vec[:,shift:shift+nmodes]
    val = val[shift:shift+nmodes]
    return vec,val

In [161]:
def hess_to_eig_torch(hess,nmodes=3, shift=5):
    """
    hess, shape: 3*n_nodes, 3*n_nodes
    return 
      vec, shape 3*n_nodes,n_nodes
        first mode (column) is most dominant
    """
    val_gpu, vec_gpu = torch.linalg.eigh(hess)
    vec,val = vec_gpu, val_gpu
    vec = vec[:,shift:shift+nmodes]
    val = val[shift:shift+nmodes]
    return vec,val

In [143]:
n=10
np.random.seed(0)
hess = np.random.normal(np.zeros((n,n)),1)
hess = hess + hess.T

vec_skcuda, val_skcuda = hess_to_eig_gpu(hess,nmodes=3, shift=5)

n=10
np.random.seed(0)
hess = np.random.normal(np.zeros((n,n)),1)
hess = hess + hess.T
vec_torch, val_torch = hess_to_eig_torch(tensor(hess),nmodes=3, shift=5)

In [162]:
np.allclose(val_skcuda,val_torch.numpy())

True

In [163]:
np.allclose(vec_skcuda,vec_torch.numpy(),atol=1e-4)

True